In [1]:
import tensorflow as tf
try: [tf.config.experimental.set_memory_growth(gpu,True) for gpu in tf.config.list_physical_devices('GPU')]
except: pass

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from mltu.dataProvider import DataProvider
from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer,LabelPadding
from mltu.augmentors import RandomBrightness, RandomRotate,RandomErodeDilate
from mltu.losses import CTCloss
from mltu.callbacks import Model2onnx,TrainLogger
from mltu.metrics import CWERMetric
from model import train_model
from configs import ModelConfigs
import stow
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

In [2]:
def download_and_unzip(url,extract_to='Datasets'):
    http_respose=urlopen(url)
    zipfile=ZipFile(BytesIO(http_respose.read()))
    zipfile.extractall(path=extract_to)

if not stow.exists(stow.join('Datasets','captcha_images_v2')):
    download_and_unzip('https://github.com/AakashKumarNain/CaptchaCracker/raw/master/captcha_images_v2.zip')


In [13]:
dataset, vocab, max_len = [], set(), 0
for file in stow.ls(stow.join('Datasets', 'captcha_images_v2')):
    dataset.append([stow.relpath(file), file.name])
    vocab.update(list(file.name))
    max_len = max(max_len, len(file.name))

In [14]:

configs = ModelConfigs()

# Save vocab and maximum text length to configs
configs.vocab = "".join(vocab)
configs.max_text_length = max_len
configs.save()


In [15]:
data_provider = DataProvider(
    dataset=dataset,
    skip_validation=True,
    batch_size=configs.batch_size,
    data_preprocessors=[ImageReader()],
    transformers=[
        ImageResizer(configs.width, configs.height),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab))
        ],
)

2024-06-22 16:39:24,879 INFO mltu.dataProvider: Skipping Dataset validation...


In [16]:
train_data_provider,val_data_provider=data_provider.split(split=0.9
                                                          )
train_data_provider.augmentors=[RandomBrightness(),RandomRotate(),RandomErodeDilate()]
model=train_model(
    input_dim=(configs.height,configs.width,3),
    output_dim=len(configs.vocab),)

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=configs.learning_rate), 
    loss=CTCloss(), 
    metrics=[CWERMetric()],
)


In [18]:
model.summary(line_length=110)
stow.mkdir(configs.model_path)

Model: "model_1"
______________________________________________________________________________________________________________
 Layer (type)                       Output Shape            Param #      Connected to                         
 input (InputLayer)                 [(None, 50, 200, 3)]    0            []                                   
                                                                                                              
 lambda_1 (Lambda)                  (None, 50, 200, 3)      0            ['input[0][0]']                      
                                                                                                              
 conv2d_24 (Conv2D)                 (None, 50, 200, 16)     448          ['lambda_1[0][0]']                   
                                                                                                              
 batch_normalization_18 (BatchNorma  (None, 50, 200, 16)    64           ['conv2d_24[0][0]']   

<stow.Directory: \Users\darre\OneDrive\Desktop\Darren\College Stuff\Image Captcha recognition\Models\02_captcha_to_text\202406221639>

In [19]:
earlystopper = EarlyStopping(monitor='val_CER', patience=40, verbose=1)
checkpoint = ModelCheckpoint(f"{configs.model_path}/model.h5", monitor='val_CER', verbose=1, save_best_only=True, mode='min')
trainLogger = TrainLogger(configs.model_path)
tb_callback = TensorBoard(f'{configs.model_path}/logs', update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_CER', factor=0.9, min_delta=1e-10, patience=20, verbose=1, mode='auto')
model2onnx = Model2onnx(f"{configs.model_path}/model.h5")

In [23]:
model.fit(
    train_data_provider,
    validation_data=val_data_provider,
    epochs=configs.train_epochs,
    callbacks=[earlystopper, checkpoint, trainLogger, reduceLROnPlat, tb_callback, model2onnx],
    workers=configs.train_workers
)


Epoch 1/1000
15/15 [==============================] - ETA: 0s - loss: 0.1615 - CER: 0.0094 - WER: 0.0470
Epoch 1: val_CER did not improve from 0.01346


2024-06-22 18:27:43,553 INFO root: Epoch 0; loss: 0.16152510046958923; CER: 0.009188035503029823; WER: 0.044871795922517776; val_loss: 0.5801244974136353; val_CER: 0.023076923564076424; val_WER: 0.10576923191547394


15/15 [==============================] - 30s 2s/step - loss: 0.1615 - CER: 0.0094 - WER: 0.0469 - val_loss: 0.5801 - val_CER: 0.0231 - val_WER: 0.1058 - lr: 7.2900e-04
Epoch 2/1000
15/15 [==============================] - ETA: 0s - loss: 0.1343 - CER: 0.0053 - WER: 0.0267
Epoch 2: val_CER did not improve from 0.01346


2024-06-22 18:28:13,245 INFO root: Epoch 1; loss: 0.13431957364082336; CER: 0.00555555522441864; WER: 0.02777777798473835; val_loss: 0.6977093815803528; val_CER: 0.02500000223517418; val_WER: 0.11538461595773697


15/15 [==============================] - 30s 2s/step - loss: 0.1343 - CER: 0.0054 - WER: 0.0268 - val_loss: 0.6977 - val_CER: 0.0250 - val_WER: 0.1154 - lr: 7.2900e-04
Epoch 3/1000
15/15 [==============================] - ETA: 0s - loss: 0.1298 - CER: 0.0084 - WER: 0.0407
Epoch 3: val_CER did not improve from 0.01346


...

2024-06-22 19:07:53,907 INFO root: Epoch 87; loss: 0.043394893407821655; CER: 0.003205128712579608; WER: 0.016025641933083534; val_loss: 0.4894173741340637; val_CER: 0.017307691276073456; val_WER: 0.07692307978868484


15/15 [==============================] - 27s 2s/step - loss: 0.0434 - CER: 0.0023 - WER: 0.0115 - val_loss: 0.4894 - val_CER: 0.0173 - val_WER: 0.0769 - lr: 5.9049e-04
Epoch 89/1000
15/15 [==============================] - ETA: 0s - loss: 0.0460 - CER: 0.0020 - WER: 0.0102
Epoch 89: val_CER did not improve from 0.01154


2024-06-22 19:08:22,279 INFO root: Epoch 88; loss: 0.0459783673286438; CER: 0.002136752475053072; WER: 0.010683760978281498; val_loss: 0.6325829029083252; val_CER: 0.015384615398943424; val_WER: 0.06730769574642181



Epoch 89: ReduceLROnPlateau reducing learning rate to 0.0005314410547725857.
15/15 [==============================] - 28s 2s/step - loss: 0.0460 - CER: 0.0021 - WER: 0.0103 - val_loss: 0.6326 - val_CER: 0.0154 - val_WER: 0.0673 - lr: 5.9049e-04
Epoch 89: early stopping


2024-06-22 19:08:23,950 INFO tf2onnx.tfonnx: Using tensorflow=2.10.1, onnx=1.12.0, tf2onnx=1.14.0/8f8d49
2024-06-22 19:08:23,950 INFO tf2onnx.tfonnx: Using opset <onnx, 15>
2024-06-22 19:08:24,088 INFO tf2onnx.tf_utils: Computed 0 values for constant folding
2024-06-22 19:08:24,096 INFO tf2onnx.tf_utils: Computed 0 values for constant folding
2024-06-22 19:08:24,099 INFO tf2onnx.tf_utils: Computed 0 values for constant folding
2024-06-22 19:08:24,102 INFO tf2onnx.tf_utils: Computed 0 values for constant folding
2024-06-22 19:08:24,152 INFO tf2onnx.tf_utils: Computed 2 values for constant folding
2024-06-22 19:08:24,235 INFO tf2onnx.tfonnx: folding node using tf type=StridedSlice, name=model_1/bidirectional_1/forward_lstm_1/PartitionedCall/strided_slice
2024-06-22 19:08:24,236 INFO tf2onnx.tfonnx: folding node using tf type=StridedSlice, name=model_1/bidirectional_1/backward_lstm_1/PartitionedCall/strided_slice
2024-06-22 19:08:24,395 INFO tf2onnx.optimizer: Optimizing ONNX model
2024-0

In [24]:
train_data_provider.to_csv(stow.join(configs.model_path,'train.csv'))
val_data_provider.to_csv(stow.join(configs.model_path,'val.csv'))